# Pre-processing data

In this notebook we talk about some of the functions and methods that are commonly used to deal with numeric and categorical columns in our data in preparation to doing machine learning. Without properly pre-processing the data, our ML algorithm will either fail to execute or execute but return results that are incorrect or incomplete. Since we haven't introduced machine learning yet, it's too early to show examples of how this can happen, but we will see examples in future lectures.

As usual, let's begin by reading some data. We want to read the data and find some ways to summarize the data.

In [ ]:
import pandas as pd
import numpy as np
# you need to run this to produce visualizations in a jupyter notebook
%matplotlib inline
pd.options.display.max_columns = 40
pd.options.display.max_rows = 100
pd.options.display.precision = 3

LocalFile = '../../data/auto-mpg.csv'
UCI_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
url = UCI_url
auto = pd.read_csv(url, sep = '\s+', header = None, 
                   names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 
                            'acceleration', 'model year', 'origin', 'car_name'])
auto.head()

Let's check the dataframe size and column data types:

In [ ]:
display(auto.shape)
display(auto.dtypes)

We can see that we have quite a few numeric columns in here. Numeric columns can be **integer**, **float**, or **datetime** columns. However, we also need to be careful: a column like `cylinders` looks numeric, but probably should be treated as a categorical column. Why? Ask yourself which one of these two is more likely:

- do any kind of math with `cylinders`, such as taking the log of it, adding it to another column, or being interested in knowing the average number of cylinders?
- use cylinder as a grouping variable, such as wanting to see the average horsepower **by cylinder**?

We can probably agree that the second case is more likely. For this reason, we need to think of these types of variables as categorical despite them having numeric values.

Let's now look at some distributions for our numeric columns.

In [ ]:
import seaborn as sns

sns.pairplot(auto.sample(100),  kind='scatter', plot_kws={'alpha':0.25});

None of the feature seem to have any extreme values. Let's check for missing values. To see if any columns in the data are missing, we can use the `isnull` method, followed by `any` where the `axis = 0` is used to ask if **any** of values **across columns** is null: `False` means that none is missing, and `True` means that at least one value is missing.

In [ ]:
auto.isnull().any(axis = 0)

Even better would be to use `sum` to get the count of values that are missing for each column.

In [ ]:
auto.isnull().sum(axis = 0)

Note that all of this would work only if the missing data is properly marked as null. Often when reading external data sources, other conventions are used to mark data as missing, such as using `-999` for numeric data, or the string `NA` or `?` or the empty string for categorical data. Such values by default would not be picked up by `isnull`. So we would need to first recode them as `np.nan`.

### Exercise (8 minutes)

Let's illustrate the above point with an example, let's intentially introduce a few missing values in the data, but using characters that won't be recognized as such.

In [ ]:
auto.loc[2, ['mpg', 'car_name']] = [-999, ""] # make mpg and car_name in the 3rd row NA
# Add zero to origin of 4th car
auto.head()

We look for the following occurences to detect the missing values:
- inappropriate data types
- the length of strings in `car_name`
- outliers in numerical values
- A numerical value of zero (0) can be a stand-in for null.
- very common or very rare values:  Check number of unique values and if reasonable, review the values
- Unusual characters like question mark ("?"), blank (" "), empty (""), "x", "-"
  
How can we use a boolean indexer in `.loc` of `pandas` to find the missing values?

In [ ]:
# Add code here 001
auto.loc[auto['mpg'] < 0,'mpg'], auto.loc[auto['car_name'] == "",'car_name']
# for row in auto.iterrows():
#     if row[1] < 0:
#         print(row)

In [ ]:
for t in auto.dtypes.items():
    print(t)

In [ ]:
num_cols = [col for col,type in auto.dtypes.items() if type != 'object']
num_cols

In [ ]:
(auto.select_dtypes(['int64','float64']) < 0).any()

- Replace the above missing values with `np.nan`

In [ ]:
# Add code here 002
auto.loc[auto['mpg'] < 0,'mpg'] = np.nan
auto.loc[auto['car_name'] == "",'car_name'] = np.nan

- Now show any rows in the data that have any missing values. HINT: you will need to use `any` with `axis = 1`.

In [ ]:
auto.head()

In [ ]:
# Add code here 003
auto.loc[auto.isnull().any(axis=1),:]

We also can see from the scatter plot matrix that `weight` and `displacement` have a **positive relationship**.  A positive relationship means that as one goes up, so does the other. We may wonder if `cylinders` plays a part.

- Plot the scatter plot between `weight` and `displacement`, color-coded by `cylinders`.

In [ ]:
# Add code here 004

- Obtain some basic summary statistics for the columns in `auto`. HINT: Use `describe`.

In [ ]:
# Add code here 005

Notice that the result of `describe` itself looks looks like a `pandas` `DataFrame` or `Series`. That means that if we wanted to extract certain pieces out of it, we should be able to do it using the methods we learned for working with `DataFrame` objects.

- Extract the mean and standard deviation for acceleration from the results above and store them in variables called `acc_mean` and `acc_std`. Then print a statement that states what the mean and standard deviations are.

In [ ]:
# Add code here 006

### End of exercise

There are other functions we can use to get additional summary statistics from the data. For example, notice that `describe` only shows us the 25th, 50th, and 75th **percentiles**. The $p$th percentile is **a value such that $p$ percent of the data is below that value**, which means the remaining $1-p$ percent of the data is above that value. For example, if the 25th percentile for acceleration `mpg` is 17.5, then 25 percent of cars in our data have `mpg` below 17.0. The 50th percentile is also known as the **median**.

Looking at percentiles helps us get a feel for the distribution of the data. For example, looking at very low or very high percentiles can help us identify **outliers** or **extreme values**. We can use the `quantile` method to get any percentile we want:

In [ ]:
auto.mpg.quantile(q = [0, 0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99, 1])

Now let's see how we can obtain these same summary statistics, but grouped by `cylinders`. There is already one way we know how to do that: we can first subset the data by `cylinders == 3` and run the summary statistic, then do the same for `cylinders == 4`, etc.

In [ ]:
auto.loc[auto['cylinders'] == 3, :].describe()

But of course the above way is tedious. Instead, we can use the `groupby` method to do it all at once. For example, here's the average of each column grouped by `cylinders`.

In [ ]:
auto.groupby('cylinders').mpg.mean()

A slightly confusing thing happens if we use `describe` instead of `mean`:

In [ ]:
results_all = auto.groupby('cylinders').describe()
print("dataframe with multi-level columns:")
display(results_all)

The problem with the above table is that there's a lot of information packed in it. Each value of `cylinders` has its own row, which is fine. But the columns look like they have a hierarchical structure, which makes sense because we asked for many summary statistics for each column.

A pandas series is a 1-D collection of scalars.  A typical pandas dataframe is a 2-D collection of scalars.  `results_all` is a multi-level dataframe, which is a 3D collection of scalars.  The 3 Dimensions are:
1. Cylinder Values: [3, 4, 5, 6, 8]
2. Numeric Columns (except Cylinder): [mpg, displacement, weight, acceleration, model, year, origin]
3. Descriptive Statistics:  [count, mean, std, min, 25%, 50%, 75%, max]

So what if for example we wanted to look at the results for `acceleration` only. Turns out we can index the multi-level dataframe the same way we usually do:

In [ ]:
print("One slice of a multi-level dataframe is a typical dataframe:")
display(results_all['acceleration'])

The difference with before is that `results_all['acceleration']` isn't just a single column, but is itself a `DataFrame`. That's because `results_all` has hierarchical columns, so its columns are themselves `DataFrame` objects. So if we want to drill further down to a specific column, such as `std`, we need to go one level deeper to get the series for the standard deviations of acceleration by cylinder.  We can even drill down one more level and request the standard deviation of acceleration for 3-cylinder vehicles.

In [ ]:
print("series for the standard deviations of acceleration by cylinder:")
display(results_all['acceleration']['std'])
print("###################")
print("scalar for the standard deviation of acceleration for 3-cylinder vehicles:")
display(results_all['acceleration']['std'][3])

So far it looks like hierarchical columns are not that complicated. However, things do get a little confusing when we want to cross the hierarchy the other way around. For example, let's say we want to quickly compare the standard deviations of all our columns, grouped by `cylinders`. This information is in the above table, but how do we extract it?

It turns out that to do this, we need to use the `IndexSlice` function in `pandas`. Here how it works:
- we use `loc` to say which rows and columns we want
- we use `slice(None)` to say we want everything at a given hierarchy
- if we want to filter at any level of hierarchy, we provide the index we want to filter by

In [ ]:
from pandas import IndexSlice as idx
results_all.loc[: , idx[:, 'std']] # results_all.loc[: , idx[:, ['std', 'mean']]]

By the way, the same way that we can have hierarchical columns, we can also have hierarchical rows, more specifically hierachical row indexes. Examine the result generated by the cell below. Do see the hierarchical structure of the row index?

In [ ]:
percentiles = np.arange(0, 1, 0.1)
auto.groupby('cylinders')[['mpg', 'displacement']].quantile(q = percentiles)

Hierarchical indexes are called `MultiIndex` in `pandas`. This is a more advanced topic and we will leave it at that for now.

### Exercise (7 minutes)

We saw how a histogram shows us counts for **evenly-sized** intervals of a numeric column. But what if we wanted to see counts for any intervals we specify, evenly-sized or not? Here's an example:

- Find out how many cars have `mpg` less than 18, between 18 and 25, and 25 or more? HINT: There are many ways of doing this, so to narrow it down a little do this:
  - use `loc` to limit data to the above ranges
  - use the `shape` method to get row counts

In [ ]:
# Add code here 007
autos_under18 = auto[auto.mpg < 18]
autos_under25 = auto[(auto.mpg > 18) & (auto.mpg < 25)]
autos_over25 = auto[auto.mpg > 25]

print(autos_over25.shape,autos_under25.shape,autos_under18.shape)

- How many rows were not counted because they did not fall into any of these intervals?

In [ ]:
# Add code here 008
rows_count = autos_over25.shape[0] + autos_under25.shape[0] + autos_under18.shape[0]
rows_lost = auto.shape[0] - rows_count

print(f"total rows are {rows_count} compared to {auto.shape[0]} thus we lost {rows_lost}")

There is another, much faster way to solve the exercise above: 

- Use the `pd.cut` function to bin `mpg` and specify `[0, 18, 25, 100]` as the cut-offs (the `bins` argument). Store the results as a new column in the data called `mpg_cat`.

In [ ]:
# Add code here 009
mpg_cat = pd.cut(auto['mpg'], [0,18,25,100])

- Use `value_counts` to get counts for `mpg_cat`.

In [ ]:
# Add code here 010
mpg_cat.value_counts()

- Create `mpg_cat` again, but make the following changes to it: 
  - Note that the choice of using 0 and 100 as the lower and upper bounds is a little arbitrary. So use `-np.Inf` and `np.Inf` instead.
  - By default, cut labels the catgories based on the interval range it covers, but we can change the labels to anything we like using the `labels` argument. Rename the labels to `low`, `med` and `high`.

In [ ]:
# Add code here 011
mpg_cat = pd.cut(auto.mpg, [-np.inf,18,25,np.inf])
mpg_cat.value_counts()

In [ ]:
mpg_cat.value_counts().plot.bar()

### Feature Summarization versus Feature Transformation
The `pd.cut` function is different from the other functions (or methods) we learned about so far. The `describe`, `mean`, `quantile`, or `value_counts` functions are **feature summarization** functions, but `pd.cut` is a **feature transformation** function, meaning that not only its input, but also its output are features. In the above example, the input to `pd.cut` was `mpg`, and the output was the `mpg_cat` which we chose to append to the data as a new column, but could have also stored in a separate variable. Storing it as a separate variable is a good idea if the variable stores some intermediate values that we use as part of a transformation but don't need anymore afterwards. 

### End of exercise

## Working with categorical data

Let's now look at common tasks for treating **categorical data** prior to modeling. Categorical data needs a lot of attention during data pre-processing. This is because most machine learning algorithms don't deal directly with categorical data. Instead we need to **recode** the data from categorical into numeric, and we will see how we do that in this notebook.

Let's begin by reading some data. We will use a marketing data set of bank customers. You can read more about the data [here](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing). 

In [ ]:
import pandas as pd
import numpy as np

bank = pd.read_csv('../../data/bank-full.csv', sep = ";")
bank.head()

We can see that our data contains many categorical columns, including the target itself. Let's check the data types:

In [ ]:
bank.info()

We can use the `select_dtypes` method to limit the data to just the categorical columns.

In [ ]:
bank.select_dtypes('object').head() # bank.loc[:, bank.dtypes == 'object'].head()

It turns out there are **two kinds of data types for categorical data** in `pandas`: `object` and `category`. By default, any non-numeric column will inherit the `object` type, but we can later convert it to `category` type. An `object` column type can be modified at will, but a `catogory` type is only appropriate for a column with **a limited number pre-defined categories**. This is because the `category` type is a more rigid data type with additional limitations on what they can store. So this only makes sense when the categories are known and few. Let's illustrate that by turning some of the columns in our data into a `category` columns.

In [ ]:
cat_cols = ['marital', 'default', 'housing', 'loan']
bank[cat_cols] = bank[cat_cols].astype('category')

Why would we want to add additional rigidity? Because this way we can impose some amount of **data integrity**. For example, if `marital` should always be limited to "single", "divorced" or "married" then by converting `marital` into a `category` column we can prevent the data from introducing any other category without first adding it as one of the acceptable categories for this column.

In [ ]:
bank['marital'].cat.categories

### Exercise (6 minutes)

- Try to change the `marital` column at the second row to the value "widowed". You should get an error.

In [ ]:
print('Is widowed a category in the marital column?  ', 'widowed' in bank['marital'].cat.categories)
# Uncomment the following line of code:
# bank.loc[1, 'marital'] = "widowed"

- To fix the error, you need to add "widowed" as one of the acceptable categories. Use the `cat.add_categories` method to add "widowed" as a category and then try again to make sure you don't get an error this time.

In [ ]:
# Add code here 012
bank['marital'] = bank['marital'].cat.add_categories('widowed')
bank.loc[1,'marital'] = 'widowed'
bank.head()

- Undo your change by reassigning the `marital` column at the second row to the value "single". Get a count of unique values for `marital` now. Do you notice anything? Explain what and why?

In [ ]:
# Add code here 013
bank.loc[1,'marital'] = 'single'

Categorical columns have other useful methods, and their names speak for themselves, such as
`as_ordered`, `as_unordered`, `remove_categories`, `remove_unused_categories`, `rename_categories`, `reorder_categories`, and `set_categories`. It is important to be aware of this functionality and use it when it makes sense. Of course an alternative to using these is to convert the column back to `object` and make all the changes we want and then turn it back into `category`, but using the above methods makes the code "cleaner".

So we saw that a `category` column has pre-defined categories and a set of methods specific to itself for changing the categories, whereas an `object` column is more a type of **free-form** column where any value is acceptable. One way the above distinction matters if when we need to rename the categories for a categorical column. Changing the categories of a categorical column is an example of **recoding** or **remapping**.

- Let's first begin with an example using `job`, which has type `object`. Rename the category "management" to "managerial". HINT: find all rows where `job` is the string `'management'`, and use `loc` to change those rows to the string `'managerial'`.

In [ ]:
# Add code here 014
bank.loc[bank.job == 'management', 'job'] = 'managerial'

The above approach works fine, but it's tedious if we have a lot of changes we want to make. The better way to do it is to create a Python dictionary that maps old values (values we want to change) to new values, then use the `replace` method to replace them all at once.

- Create such a dictionary and use `replace` to make the following changes in the `job` column:

  - rename `'student'` to `'in-school'`
  - consolidate `'housemaid'` and `'services'` into a single group called `'catering'`
  - change `unknown` to a missing value, i.e. `np.NaN` (without quotes)

In [ ]:
# Add code here 015
ReplacementMap = {'student':'in-school', 'housemaid':'catering', 'services':'catering', "unknown":np.nan}
bank.job.replace(ReplacementMap, inplace = True)

- The above transformation was a renaming and a consolidation (grouping, binning) of catgories!

- Get a count of unique values for `job` to make sure everything worked. Note that `value_counts()` does not provide count for missing values by default. We need to specify `dropna = False` to include missing vaules in the count.

In [ ]:
# Add code here 016
bank.job.value_counts(dropna = False)

### End of exercise

The `replace` method works equally well with a column of type `category`, however **it changes its type to `object`!** So either we have to convert it back to `category`, or we need to use the `rename_categories` method to replace values, which workes very similarly to `replace`. Here's an example:

In [ ]:
bank['marital'] = bank['marital'].cat.rename_categories({'married': 'taken'})
bank['marital'].value_counts()

Categorical columns can also be easily generated from numeric columns. For example, let's say we want to have a column called `high_balance` that is `True` when balance exceeds $2,000 and `False` otherwise. Technically this would be a boolean column, but in practice it acts as categorical column. Generating such a column is very easy. We refer to such binary colums as **dummy variables** or **flags** because they single out a group.

In [ ]:
bank['high_balance'] = bank['balance'] > 2000

In [ ]:
bank.head()

The process of creating a dummy variable **for each category** of a categorical feature is called **one-hot encoding**. Let's see what happens if we one-hot-encode `marital`.

In [ ]:
bank['marital_taken'] = (bank['marital'] == 'taken').astype('int')
bank['marital_single'] = (bank['marital'] == 'single').astype('int')
bank['marital_divorced'] = (bank['marital'] == 'divorced').astype('int')

In [ ]:
bank.filter(like = 'marital').head()

One-hot encoding is a common enough task that we don't need to do it manually like we did above. Instead we can use `pd.get_dummies` to do it in one go.

In [ ]:
pd.get_dummies(bank[['marital', 'job']], prefix = ['marital', 'job']).head()

There's an even more streamlined way to do one-hot encoding, although at first blush it appears less straight-forward, but there is a reason it is set up this way and we will explain that later. Just like normalization, one-hot-encoding is a common pre-processing task and we can turn to the `sklearn` library to do the hard part for us.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

bank_cat = bank.select_dtypes('category').copy() # only select columns that have type 'category'
onehot = OneHotEncoder(sparse_output = False) # initialize one-hot-encoder
onehot.fit(bank_cat)
col_names = onehot.get_feature_names_out(bank_cat.columns) # this allows us to properly name columns
bank_onehot =  pd.DataFrame(onehot.transform(bank_cat), columns = col_names)
bank_onehot.head()

So we can see that one-hot encoding created a **binary feature** for **each category of each categorical column** in the data. Although to be more specific, we limited it to columns whose type is `category` and excluded columns whose type is `object`. This is because one-hot encoding can quickly blow up the number of columns in the data if we are not careful and include categorical columns with lots of categories (also called **high-cardinality** categorical columns). In such cases, we would one-hot encode only the top $n$ categories (what $n$ should be and what "top" should mean depends on the context). Another 

What is the point of doing this? The reason we do this is that most machine learning algorithms do not work **directly** with categorical data, so we need to encode the categorical data which turns it into numeric data. One-hot encoding is just one type of encoding, but it is the most common one.

One last note about the `sklearn` pre-processing transformations we learned about in this notebook: If you look at examples online, you may notice that instead of calling `fit` and `transform` separately, you can call `fit_transform` which combines the two steps into one. This may seem reasonable and saves you one extra line of code, but we discourage it. The following exercise will illustrate why, but the main reason will become clear when we talk about machine learning.

### Exercise (5 minutes)

We want to one-hot-encode the `education` and `month` columns in bank. 

- First create an instance of the one-hot encoder and invoke fit and transform with fit_transform using the `education` and `month` columns in bank.

In [ ]:
# Add code here 017
cols = ['education','month']
binary_maker = OneHotEncoder(sparse_output=False)
binary_data = binary_maker.

Create new "hybrid" column names for the new one-hot-encoded values.  The column names are a hybrid of the original column name and the column values.

In [ ]:
# Add code here 018

Create a new dataframe out of the one-hot-encoded values and the new column names

In [ ]:
# Add code here 019

Concatenate the new dataframe of the new one-hot-encoded columns to the original bank dataframe and then drop the original `education` and `month` columns

In [ ]:
# Add code here 020

### End of exercise

One final note about one-hot encoding: there are more sophisticated way that we can perform one-hot encoding by using hash functions. We leave the reader to read about **feature hashing** (if you know what **hash functions** are, then feature hashing is very easy to grasp). This method has the advantage that we do not need to know what the categories are ahead of time. Not only does this free us from keeping track of the categories, but it also means that we don't have to load the pre-defined list of categories with the model **at score time** when the model is deployed: We just use the hash function to compute it on the fly. This can make scoring much more efficient when the size of features is large.

# Assignment
The assignment is in **Lesson_05_b_assignment.ipynb**